In [3]:
import pandas as pd

data = pd.read_csv('linktable.csv',index_col=0)
transcripts = data[data['Hyperlink'].str.contains('TRANS')]
testdata = transcripts[:10]

In [ ]:
import requests as req

# Takes our link table:
#   FilingName; FilingDate; AENumber; Attachment; Hyperlink

for index, row in testdata.iterrows():
    url = row['Hyperlink']
    filename = 'transcripts/' + url[(url.find('-')+1):]

    try:
        response = req.get(url)
        response.raise_for_status()

        with open(filename, 'wb') as file:
            file.write(response.content)
        print("file downloaded successfully")
    except req.exceptions.RequestException as e:
        print(f'Error downloading file {index}: {e}')

In [21]:
from pypdf import PdfReader
import os

files = os.listdir('transcripts/')
for file in files:
    if file.endswith('.pdf'):
        reader = PdfReader('transcripts/'+file)
        page_texts = []

        for page in reader.pages:
            text = page.extract_text()
            lines = text.split('\n')
            lines = lines[3:-2]
            cleaned_lines = []

            for line in lines:
                cleaned_line = line.split(' ')[:-2]
                cleaned_line = ' '.join(cleaned_line)
                cleaned_lines.append(cleaned_line)

            cleaned_text = '\n'.join(cleaned_lines)+'\n'
            page_texts.append(cleaned_text)

        with open('transcripts/'+file[:-3]+'txt', 'w') as f:
            for page in page_texts:
                f.write(page)